In [17]:
# pre-requisites 

! pip install python-dotenv > /dev/null
print("installed python-dotenv")
! pip install transformers torch huggingface_hub torchvision datasets> /dev/null
print("installed transformers, torch, huggingface_hub, datasets and torchvision")

# Imports 
import pandas as pd 
import os 
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset 
from huggingface_hub import login

#ENV Variables 
TOKEN = os.getenv("TOKEN")
TOKENIZERS_PARALLELISM = os.getenv("TOKENIZERS_PARALLELISM")
DATASET_PATH = 'fol_dataset.csv' 
TEST_SIZE = 0.2
SEED = 42

MODEL_LIST = [
    "google/gemma-3-1b-it"
]
login(token = TOKEN)

print("Initialised environment variables")

installed python-dotenv
installed transformers, torch, huggingface_hub, datasets and torchvision
Initialised environment variables


Writing the predefinded functions in the cell below

In [18]:
def preprocess(batch):
     inputs = tokenizer(
         batch['expression'],
         max_length=128,
         truncation=True,
         padding = 'max_length',
         
     )    
    
     labels = tokenizer(
         batch['english_description'],
         max_length =128,
         truncation = True, 
         padding = 'max_length'
     )
     inputs['labels']=labels['input_ids']
     return inputs

In [ ]:
# Running the training loop in order to train the model 

for model in MODEL_LIST:
    
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForCausalLM.from_pretrained(model)
    
    dataset = load_dataset('csv', data_files = DATASET_PATH )['train']
    split_dataset = dataset.train_test_split(test_size = TEST_SIZE, seed = SEED)
    
    train_set = split_dataset['train']
    test_set = split_dataset['test']

    train_tokenized = train_set.map(preprocess, batched=True)
    test_tokenized = test_set.map(preprocess, batched=True)
    
    training_args = TrainingArguments(
        
            output_dir = "./results",
            per_device_train_batch_size = 12,
            per_device_eval_batch_size = 12,
            logging_steps = 2,
            eval_steps = 2, 
            num_train_epochs = 5,
            learning_rate = 2e-5,
    
    )

    trainer = Trainer(
        
            model = model,
            args = training_args,
            train_dataset = train_tokenized,
            eval_dataset = test_tokenized,
    
    )

    trainer.train()
    model.save_pretrained("/usr3/allanp/llm_reasoning_notebook_experiments/model_ckpts", from_pt=True) 
    
    

In [ ]:

# df=pd.read_csv("fol_dataset_reversed.csv")
# column_titles=['english_description','expression']
# df=df.reindex(columns=column_titles)
# df.to_csv('fol_dataset.csv', index = False)

In [ ]:
import json

logs = trainer.state.log_history
for i in logs:
  print(i["loss"])
      

18.6425
8.9203
12.8907
6.6149
2.9383
2.0198
1.7966
1.3202
1.2446
1.2575
0.9901
4.6749
1.3118
0.991
1.949
1.1092
0.9036
1.0823
0.9094
0.8511
1.0318
0.8666
0.8301
0.9548
0.7413
0.7665
0.8438
0.7715
0.7512
0.8163
0.7295
0.7204
0.7777
0.7623
0.7137
0.7258
0.7284
0.6819
0.7513
0.6791
0.7529
0.7537
0.6459
0.7161
0.7104
0.7206
0.6648
0.7008
0.7144
0.7071
0.6703
0.7969
0.6995
0.6849
0.6491
0.6585
0.6163
0.7323
0.6734
0.6826
0.7186
0.6943
0.6966
0.7062
0.6474
0.6555
0.6614
0.7018
0.6718
0.7294
0.6611
0.6378
0.6617
0.6446
0.6079
0.642
0.6485
0.6517
0.6152
0.6194
0.706
0.6094
0.6619
0.6439
0.6593
0.6395
0.6374
0.5931
0.6309
0.614
0.6303
0.6197
0.6912
0.659
0.6315
0.6144
0.6694
0.6643
0.6514
0.7025
0.6387
0.6401
0.6561
0.6387
0.6094
0.6603
0.6408
0.6396
0.6049
0.5995
0.6493
0.6205
0.628
0.6828
0.6436
0.6441
0.6775
0.6771
0.6473
0.6683
0.6177
0.6811
0.6318
0.6288
0.6725
0.6505
0.668
0.6403
0.6301
0.6369
0.6644
0.6107
0.6057
0.6137
0.6526
0.6562
0.6942
0.5895
0.6953
0.6731
0.6482
0.648
0.6457
0.6514

KeyError: 'loss'